In [1]:
import os
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'
import argparse
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from scipy import stats
from src.utils.utilities import *
from src.metrics.swap import SWAP
from src.datasets.utilities import get_datasets
from src.search_space.networks import *

# Settings for console outputs
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [2]:
data_path = "datasets"
seed = 0
device = "cpu"
repeats = 32
input_samples = 16

In [2]:
# parser = argparse.ArgumentParser()

# # general setting
# parser.add_argument('--data_path', default="datasets", type=str, nargs='?', help='path to the image dataset (datasets or datasets/ILSVRC/Data/CLS-LOC)')
# parser.add_argument('--seed', default=0, type=int, help='random seed')
# parser.add_argument('--device', default="cpu", type=str, nargs='?', help='setup device (cpu, mps or cuda)')
# parser.add_argument('--repeats', default=32, type=int, nargs='?', help='times of calculating the training-free metric')
# parser.add_argument('--input_samples', default=16, type=int, nargs='?', help='input batch size for training-free metric')

# args = parser.parse_args()

usage: ipykernel_launcher.py [-h] [--data_path [DATA_PATH]] [--seed SEED]
                             [--device [DEVICE]] [--repeats [REPEATS]]
                             [--input_samples [INPUT_SAMPLES]]
ipykernel_launcher.py: error: unrecognized arguments: --f=c:\Users\USER\AppData\Roaming\jupyter\runtime\kernel-v2-22744y6rbtiVsW0Mb.json


SystemExit: 2

In [3]:
device = torch.device(device)

arch_info = pd.read_csv(data_path+'/DARTS_archs_CIFAR10.csv', names=['genotype', 'valid_acc'], sep=',')

train_data, _, _ = get_datasets('cifar10', data_path, (input_samples, 3, 32, 32), -1)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=input_samples, num_workers=0, pin_memory=True)
loader = iter(train_loader)
inputs, _ = next(loader)  

results = []


FileNotFoundError: [Errno 2] No such file or directory: 'datasets/DARTS_archs_CIFAR10.csv'

In [11]:
print(arch_info)

                                              genotype  valid_acc
0    Genotype(normal=[('none', 0), ('dil_conv_5x5',...  85.000000
1    Genotype(normal=[('sep_conv_5x5', 0), ('sep_co...  85.239998
2    Genotype(normal=[('dil_conv_5x5', 0), ('skip_c...  81.290001
3    Genotype(normal=[('dil_conv_5x5', 0), ('sep_co...  77.720001
4    Genotype(normal=[('none', 1), ('sep_conv_3x3',...  82.040001
..                                                 ...        ...
995  Genotype(normal=[('dil_conv_3x3', 0), ('dil_co...  74.369995
996  Genotype(normal=[('dil_conv_5x5', 1), ('max_po...  80.290001
997  Genotype(normal=[('sep_conv_3x3', 1), ('avg_po...  79.059998
998  Genotype(normal=[('sep_conv_3x3', 1), ('dil_co...  79.889999
999  Genotype(normal=[('avg_pool_3x3', 1), ('dil_co...  79.199997

[1000 rows x 2 columns]


In [ ]:
for index, i in arch_info.iterrows():
    print(f'Evaluating network: {index}')

    network = Network(3, 10, 1, eval(i.genotype))
    network = network.to(device)

    swap = SWAP(model=network, inputs=inputs, device=device, seed=seed)

    swap_score = []

    for _ in range(repeats):
        network = network.apply(network_weight_gaussian_init)
        swap.reinit()
        swap_score.append(swap.forward())
        swap.clear()

    results.append([np.mean(swap_score), i.valid_acc])

results = pd.DataFrame(results, columns=['swap_score', 'valid_acc'])
print()    
print(f'Spearman\'s Correlation Coefficient: {stats.spearmanr(results.swap_score, results.valid_acc)[0]}')


In [33]:
input_file = open("input_RGB_32_32.txt","r")
input_dim = 32
inputs = []
for input_data in input_file.readlines():
    if input_data == "":
        break

    data = input_data.split()
    data_list = []
    for color in range(3):
        color_list = []
        for i in range(input_dim):
            data_list_list = []
            for j in range(input_dim):
                data_list_list.append(int(data[color*input_dim*input_dim + i*input_dim + j]))
            color_list.append(data_list_list)
        data_list.append(color_list)

    # the input chennel
    inputs.append(data_list)
input_file.close()

In [29]:
print(len(inputs[0][2][0]))

TypeError: object of type 'int' has no len()

In [35]:
import PIL.Image as Image

INPUT_IDX = 40

def p(rows, cols, r,g,b):
    image = Image.new("RGB", (rows, cols))
    for i in range(rows):
        for j in range(cols):
            image.putpixel((i,j), (r[i][j], g[i][j], b[i][j]))
    image.show()

p(input_dim, input_dim, inputs[INPUT_IDX][0],inputs[INPUT_IDX][1],inputs[INPUT_IDX][2])



In [5]:
import PIL.Image as Image

In [6]:
input_file = open("input_RGB_32_32_10frame.txt","r")
input_dim = 32
inputs_4f = []
for input_data in input_file.readlines():
    if input_data == "":
        break

    data = input_data.split()
    data_list = []
    for color in range(3):
        color_list = []
        for i in range(input_dim):
            data_list_list = []
            for j in range(input_dim):
                data_list_list.append(int(data[color*input_dim*input_dim + i*input_dim + j]))
            color_list.append(data_list_list)
        data_list.append(color_list)

    # the input chennel
    inputs_4f.append(data_list)
input_file.close()

In [7]:
INPUT_IDX = 40

def p(rows, cols, r,g,b):
    image = Image.new("RGB", (rows, cols))
    for i in range(rows):
        for j in range(cols):
            image.putpixel((i,j), (r[i][j], g[i][j], b[i][j]))
    image.show()

p(input_dim, input_dim, inputs_4f[INPUT_IDX][0],inputs_4f[INPUT_IDX][1],inputs_4f[INPUT_IDX][2])